##### Copyright 2019 Google LLC.

Licensed under the Apache License, Version 2.0 (the "License");

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

#### Import & Utils

Install JAX, Tensorflow Datasets, and Neural Tangents

The first line specifies the version of jaxlib that we would like to import. Note, that "cp36" species the version of python (version 3.6) used by JAX. Make sure your colab kernel matches this version.

In [0]:
!pip install --upgrade -q https://storage.googleapis.com/jax-wheels/cuda$(echo $CUDA_VERSION | sed -e 's/\.//' -e 's/\..*//')/jaxlib-0.1.12-cp36-none-linux_x86_64.whl
!pip install --upgrade -q jax
!pip install -q tensorflow-datasets
!pip install -q git+https://www.github.com/google/neural-tangents

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from jax.api import jit
from jax.api import grad
from jax import random

import jax.numpy as np
from jax.experimental import stax
from jax.experimental import optimizers

import tensorflow_datasets as tfds




from neural_tangents import tangents
from neural_tangents import layers

In [0]:
def process_data(data_chunk):
  """Flatten the images and one-hot encode the labels."""
  image, label = data_chunk['image'], data_chunk['label']
  
  samples = image.shape[0]
  image = np.array(np.reshape(image, (samples, -1)), dtype=np.float32)
  image = (image - np.mean(image)) / np.std(image)
  label = np.eye(10)[label]
  
  return {'image': image, 'label': label}

# Weight Space Linearization

Setup some experiment parameters.

In [0]:
learning_rate = 1.0
batch_size = 128
training_epochs = 10
steps_per_epoch = 50000 // batch_size

Create MNIST data pipeline using TensorFlow Datasets.

In [0]:
train_data = tfds.load('mnist', split=tfds.Split.TRAIN)
train_data = tfds.as_numpy(
    train_data.shuffle(1024).batch(batch_size).repeat(training_epochs))
test_data = tfds.load('mnist', split=tfds.Split.TEST)

Create a Fully-Connected Network.

In [0]:
init_fn, f = stax.serial(
    layers.Dense(4096), 
    stax.Tanh,
    layers.Dense(10))

key = random.PRNGKey(0)
_, params = init_fn(key, (-1, 784))

Linearize the network.

In [0]:
f_lin = tangents.linearize(f, params)

Create an optimizer and initialize it for the full network and the linearized network.

In [0]:
opt_init, opt_apply, get_params = optimizers.momentum(learning_rate, 0.9)

state = opt_init(params)
lin_state = opt_init(params)

Create a cross-entropy loss function.

In [0]:
loss = lambda fx, y_hat: -np.mean(stax.logsoftmax(fx) * y_hat)

Specialize the loss to compute gradients of the network and linearized network.

In [0]:
grad_loss = jit(grad(lambda params, x, y: loss(f(params, x), y)))
grad_lin_loss = jit(grad(lambda params, x, y: loss(f_lin(params, x), y)))

Train the network and its linearization.

In [11]:
print ('Epoch\tLoss\tLinear Loss')
epoch = 0

for i, batch in enumerate(train_data):
  batch = process_data(batch)
  X, Y = batch['image'], batch['label']
  
  params = get_params(state)
  state = opt_apply(i, grad_loss(params, X, Y), state)

  lin_params = get_params(lin_state)
  lin_state = opt_apply(i, grad_lin_loss(lin_params, X, Y), lin_state)

  if i % steps_per_epoch == 0:
    print('{}\t{:.4f}\t{:.4f}'.format(
        epoch, loss(f(params, X), Y), loss(f_lin(lin_params, X), Y)))
    epoch += 1

Epoch	Loss	Linear Loss
0	0.2558	0.2558
1	0.2029	0.2029
2	0.1784	0.1784
3	0.1503	0.1503
4	0.1369	0.1369
5	0.1182	0.1183
6	0.1152	0.1153
7	0.1003	0.1005
8	0.0987	0.0989
9	0.0918	0.0919
10	0.0867	0.0869
11	0.0770	0.0773
12	0.0806	0.0808
